# Install Dependencies

In [1]:
!pip install -q transformers datasets torch seqeval evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.2 MB/s eta 0:00:00

# Env Variables

In [2]:
base_path = 'data/'

# Establish Google Drive Connection (if needed)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

base_path = 'drive/MyDrive/dataset/'

Mounted at /content/drive


# Imports

In [4]:
import json
import re
import numpy as np
import os
from datasets import Dataset
from transformers import (
    LongformerTokenizerFast,
    LongformerForTokenClassification,
    TrainingArguments,
    Trainer,
    pipeline
)
import evaluate
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split

# Helper Functions

## Load Data
Loads all json files in a specified path and combines them in one aggregated list

## Convert Numpy Floats

## Save Model Output

In [5]:
def load_json_data(folder_path):
    aggregated_data = []

    # loop through all files in the given folder
    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            with open(f"{folder_path}/{file_name}", "r") as f:
                data = json.load(f)

            aggregated_data.append(data)

    return aggregated_data

# Convert NumPy float32 to native Python floats before JSON serialization
def convert_numpy_floats(obj):
    if isinstance(obj, np.float32):
        return float(obj)
    raise TypeError

def save_model_output(output, output_path):
    with open(output_path, "w") as f:
        json.dump(output, f, ensure_ascii=False, indent=2, default=convert_numpy_floats)
    print(f"Saved validation NER predictions to {output_path}")

def compute_f1(predictions_file_path, output_path, validation_dataset):
  # Load saved predictions
  with open(predictions_file_path, 'r') as f:
      saved_preds = json.load(f)

  # Prepare gold and predicted lists
  all_gold = []
  all_pred = []
  for pred in saved_preds:
      idx = pred['index']
      gold_entities = validation_dataset[idx]['entities']
      # flatten gold mentions: (mention_text, type)
      gold_set = set()
      for ent in gold_entities:
          for m in ent['mentions']:
              gold_set.add((m, ent['type']))
      # flatten predicted mentions: pipeline outputs 'word' and 'entity_group'
      pred_list = pred['predictions']
      pred_set = set()
      for p in pred_list:
          w = p.get('word')
          et = p.get('entity_group')
          pred_set.add((w, et))
      # Append to global lists
      all_gold.append(gold_set)
      all_pred.append(pred_set)

  # Compute micro-level counts
  tp = 0
  pred_count = 0
  gold_count = 0
  for gold_set, pred_set in zip(all_gold, all_pred):
      tp += len(gold_set & pred_set)
      pred_count += len(pred_set)
      gold_count += len(gold_set)

  precision = tp / pred_count if pred_count > 0 else 0.0
  recall = tp / gold_count if gold_count > 0 else 0.0
  f1 = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0.0

  # Print and save metrics
  metrics = {
      'precision': precision,
      'recall': recall,
      'f1': f1,
      'true_positives': tp,
      'predicted': pred_count,
      'gold': gold_count
  }
  print("NER Validation Mention-level Metrics:")
  print(metrics)

  # Save metrics to JSON
  with open(output_path, 'w') as f:
      json.dump(metrics, f, indent=2)
  print(f"Saved evaluation metrics to {output_path}")

# Load data into Memory

In [6]:
# Load JSON files and store them in memory
aggregated_data = []
folder_path = f'{base_path}raw/train'

# loop through all files in the given folder
for root, dirs, files in os.walk(folder_path):
    for file_name in files:
        with open(f"{folder_path}/{file_name}", "r") as f:
            data = json.load(f)

        for d in data:
          aggregated_data.append(d)

dataset = Dataset.from_list(aggregated_data)
print("Sample example:")
print(dataset[0])

Sample example:
{'domain': 'Energy', 'title': 'Advanced_thermal_recycling_system', 'doc': 'An advanced thermal recycling system (or an ATR system) is the commercial brand name of the waste-to-energy incineration offering by Klean Power, which has been implemented in a single plant in Germany in 1999. WtE facilities such as the ATR transforms municipal solid waste (MSW) into electricity or steam for district heating or industrial customers. The combustion bottom ash, and the combustion fly ash, along with the air pollution control system fly ash, are treated to produce products that can be beneficially reused. Specifically, ATR systems consist of the following:\nSolid waste combustion, boiler and combustion control system, energy recovery and air pollution control equipment;\nCombustion bottom ash and fly ash treatment systems that produce commercially reusable products; and\nAn optional pre-processing system to recover recyclable materials contained in the MSW delivered to the facility

# Model Initialization

In [7]:
model_name = 'allenai/longformer-base-4096'

# Prepare label mappings
entity_labels = dataset[0]['entity_label_set']  # list of entity types
label_list = ['O'] + [f"B-{l}" for l in entity_labels] + [f"I-{l}" for l in entity_labels]
label2id = {l: i for i, l in enumerate(label_list)}
id2label = {i: l for l, i in label2id.items()}

# Tokenizer and model init
tokenizer = LongformerTokenizerFast.from_pretrained(model_name)
model = LongformerForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/597M [00:00<?, ?B/s]

Some weights of LongformerForTokenClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Split Data into Validation and Training Data

In [9]:
# Function to tokenize and align labels
max_length = 4096
trunc_count = 0

def tokenize_and_align_labels(example):
    global trunc_count
    encoding = tokenizer(
        example['doc'],
        return_offsets_mapping=True,
        padding='max_length',
        truncation=True,
        max_length=max_length
    )
    if len(encoding['input_ids']) == max_length:
        trunc_count += 1
    labels = [label2id['O']] * len(encoding['input_ids'])
    doc_text = example['doc']
    for ent in example['entities']:
        ent_type = ent['type']
        for m_text in ent['mentions']:
            # find all occurrences of mention string
            for match in re.finditer(re.escape(m_text), doc_text):
                start_char, end_char = match.start(), match.end()
                for idx, (off_start, off_end) in enumerate(encoding['offset_mapping']):
                    if off_start >= start_char and off_end <= end_char:
                        prefix = 'B-' if off_start == start_char else 'I-'
                        labels[idx] = label2id.get(f"{prefix}{ent_type}", label2id['O'])
    encoding.pop('offset_mapping')
    encoding['labels'] = labels
    return encoding

# Split original dataset into train and validation (preserve raw columns)
all_indices = list(range(len(dataset)))
train_idx, val_idx = train_test_split(all_indices, test_size=0.1, random_state=42)
train_orig = dataset.select(train_idx)
val_orig = dataset.select(val_idx)
print(f"Original train size: {len(train_orig)}, validation size: {len(val_orig)}")

# Tokenize & align labels separately, removing raw columns only from tokenized sets
train_tok = train_orig.map(
    tokenize_and_align_labels,
    batched=False,
    remove_columns=['domain','title','doc','triples','entities','label_set','entity_label_set']
)
val_tok = val_orig.map(
    tokenize_and_align_labels,
    batched=False,
    remove_columns=['domain','title','doc','triples','entities','label_set','entity_label_set']
)
print(f"Documents truncated in training: {trunc_count} / {len(train_tok)}")

# Use tokenized datasets for training and evaluation
train_ds = train_tok
val_ds = val_tok
print(f"Train set size: {len(train_ds)}, Validation set size: {len(val_ds)}")

Original train size: 45, validation size: 6


Map:   0%|          | 0/45 [00:00<?, ? examples/s]

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

Documents truncated in training: 51 / 45
Train set size: 45, Validation set size: 6


# Baseline NER with Untrained Model

In [10]:
# Create NER pipeline using the fine-tuned model
ner_pipe_untrained = pipeline(
    'ner',
    model=model,
    tokenizer=tokenizer,
    device=0,
    aggregation_strategy='simple'
)

# Run NER on validation documents and collect aggregated results
val_results = []
for idx, example in enumerate(val_orig):
    preds = ner_pipe_untrained(example['doc'])
    val_results.append({
        'index': idx,
        'doc_title': example.get('title', f'doc_{idx}'),
        'predictions': preds
    })

# Save to JSON in Google Drive folder
output_path = f'{base_path}processed/ner_untrained_predictions.json'
save_model_output(val_results, output_path)

scores_path = f'{base_path}processed/ner_untrained_scores.json'
compute_f1(output_path, scores_path, val_orig)

Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Input ids are automatically padded to be a multiple of `config.attention_window`: 512


Saved validation NER predictions to drive/MyDrive/dataset/processed/ner_untrained_predictions.json
NER Validation Mention-level Metrics:
{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'true_positives': 0, 'predicted': 2627, 'gold': 392}
Saved evaluation metrics to drive/MyDrive/dataset/processed/ner_untrained_scores.json


# Model Training

In [11]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./models/longformer-ner',
    num_train_epochs=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=1000
)

# Metric computation
evaluator = evaluate.load('seqeval')

def compute_metrics(p):
    predictions, labels = p
    preds = predictions.argmax(-1)
    true_labels = [[id2label[l] for l in label_seq if l != -100] for label_seq in labels]
    true_preds = [[id2label[p_] for (p_, l) in zip(pred_seq, label_seq) if l != -100]
                  for pred_seq, label_seq in zip(preds, labels)]
    results = evaluator.compute(predictions=true_preds, references=true_labels)
    return {
        'precision': results['overall_precision'],
        'recall': results['overall_recall'],
        'f1': results['overall_f1']
    }

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train!
trainer.train()

# Print final truncation info
print(f"Total truncated documents: {trunc_count} / {len(dataset)}")

<ipython-input-11-6ae34c60be7b>:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: daniellocher6 (daniellocher6-university-of-st-gallen-student-union-shsg) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,3.337400
20,2.370000
30,2.279500
40,1.276100
50,1.009200
60,1.919800
70,0.899000
80,2.539500
90,2.112600
100,0.732300


Total truncated documents: 51 / 51


# NER on Validation Data

In [12]:
# Create NER pipeline using the fine-tuned model
ner_pipe_finetuned = pipeline(
    'ner',
    model=model,
    tokenizer=tokenizer,
    device=0,
    aggregation_strategy='simple'
)

# Run NER on validation documents and collect aggregated results
val_results = []
for idx, example in enumerate(val_orig):
    preds = ner_pipe_finetuned(example['doc'])
    val_results.append({
        'index': idx,
        'doc_title': example.get('title', f'doc_{idx}'),
        'predictions': preds
    })

# Save to JSON in Google Drive folder
output_path = f'{base_path}processed/ner_trained_predictions.json'
save_model_output(val_results, output_path)

scores_path = f'{base_path}processed/ner_trained_scores.json'
compute_f1(output_path, scores_path, val_orig)

Device set to use cuda:0


Saved validation NER predictions to drive/MyDrive/dataset/processed/ner_trained_predictions.json
NER Validation Mention-level Metrics:
{'precision': 0.003236245954692557, 'recall': 0.002551020408163265, 'f1': 0.0028530670470756064, 'true_positives': 1, 'predicted': 309, 'gold': 392}
Saved evaluation metrics to drive/MyDrive/dataset/processed/ner_trained_scores.json
